In [10]:
import os
import pandas as pd
from datetime import datetime

# Ścieżki do folderów
base_path = "voiceapp"
transcript_path = os.path.join(base_path, "lista.txt")
output_path = os.path.join(base_path, "output")
summaries_folder = os.path.join(output_path, "summaries_folder")
combined_tags_folder = os.path.join(output_path, "combined_tags_folder")
final_result_folder = os.path.join(output_path, "final_result")

# Tworzenie folderu final_result, jeśli nie istnieje
os.makedirs(final_result_folder, exist_ok=True)

# Wczytanie listy transkryptów z kodowaniem UTF-8
with open(transcript_path, "r", encoding="utf-8") as file:
    transcript_files = [line.strip() for line in file]

# Lista na dane do DataFrame
data = []

# Funkcja do przeszukiwania plików w folderach
def find_files(folder, suffix):
    found_files = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith(suffix):
                found_files.append(os.path.join(root, file))
    return found_files

# Iteracja po plikach transkryptów
for transcript_path in transcript_files:
    base_name = os.path.splitext(os.path.basename(transcript_path))[0]

    # Debugowanie nazwy pliku
    print(f"Przetwarzanie: {transcript_path} (base_name: {base_name})")

    # Zmiana rozszerzenia pliku audio na .txt, aby znaleźć plik transkryptu
    transcript_txt_path = os.path.splitext(transcript_path)[0] + ".txt"
    if not os.path.exists(transcript_txt_path):
        print(f"Brak pliku transkryptu dla: {transcript_txt_path}")
        continue

    # Wczytanie zawartości transkryptu
    try:
        with open(transcript_txt_path, "r", encoding="utf-8") as file:
            transcript_content = file.read().strip()
    except FileNotFoundError:
        print(f"Nie znaleziono pliku transkryptu: {transcript_txt_path}")
        continue

    # Szukanie odpowiednich plików w summaries_folder
    summary_files = find_files(summaries_folder, f"{base_name}-summary_1.txt")
    if not summary_files:
        print(f"Brak plików summary dla: {transcript_path}")
        continue

    # Wczytanie zawartości plików summary
    for summary_path in summary_files:
        with open(summary_path, "r", encoding="utf-8") as file:
            summary_content = file.read().strip()

        # Szukanie odpowiednich plików w combined_tags_folder
        tag_files = find_files(combined_tags_folder, f"{base_name}_tags_final.csv")
        if not tag_files:
            print(f"Brak plików tags dla: {transcript_path}")
            continue

        # Wczytanie kolumny tag_name z pliku tagów
        for tag_path in tag_files:
            tags_df = pd.read_csv(tag_path)
            if "tag_name" in tags_df.columns:
                tags = tags_df["tag_name"].tolist()
            else:
                print(f"Kolumna 'tag_name' nie znaleziona w {tag_path}")
                tags = []

            # Dodanie danych do listy
            data.append({
                "reference_path": transcript_path,  # Pełna ścieżka
                "summary_content": summary_content,  # Treść podsumowania
                "tags": tags,  # Lista tagów
                "transcript_content": transcript_content,  # Treść transkryptu
            })

# Tworzenie DataFrame
final_df = pd.DataFrame(data)

# Sprawdzanie, czy wynikowy DataFrame nie jest pusty
if not final_df.empty:
    # Generowanie nazwy pliku z timestampem
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_result_filename = f"audycje_{timestamp}.csv"
    final_result_path = os.path.join(final_result_folder, final_result_filename)

    # Zapis DataFrame do pliku CSV
    final_df.to_csv(final_result_path, index=False, encoding="utf-8")
    print(f"Wynik zapisany w {final_result_path}")
else:
    print("Wynikowy DataFrame jest pusty. Sprawdź dane wejściowe.")


Przetwarzanie: C:/Users/G/Documents/GitHub/audycje.com.pl/content/audio\CJG_01_2023_01_14.mp3 (base_name: CJG_01_2023_01_14)
Przetwarzanie: C:/Users/G/Documents/GitHub/audycje.com.pl/content/audio\CJG_02_2023_01_21.mp3 (base_name: CJG_02_2023_01_21)
Przetwarzanie: C:/Users/G/Documents/GitHub/audycje.com.pl/content/audio\CJG_03_2023_01_28.mp3 (base_name: CJG_03_2023_01_28)
Przetwarzanie: C:/Users/G/Documents/GitHub/audycje.com.pl/content/audio\CJG_04_2023_02_04.mp3 (base_name: CJG_04_2023_02_04)
Przetwarzanie: C:/Users/G/Documents/GitHub/audycje.com.pl/content/audio\CJG_05_2023_02_11.mp3 (base_name: CJG_05_2023_02_11)
Przetwarzanie: C:/Users/G/Documents/GitHub/audycje.com.pl/content/audio\CJG_06_2023_02_18.mp3 (base_name: CJG_06_2023_02_18)
Wynik zapisany w voiceapp\output\final_result\audycje_20241221_004915.csv


In [5]:
print(final_df)

                                       reference_path  \
0   C:/Users/G/Documents/GitHub/audycje.com.pl/con...   
1   C:/Users/G/Documents/GitHub/audycje.com.pl/con...   
2   C:/Users/G/Documents/GitHub/audycje.com.pl/con...   
3   C:/Users/G/Documents/GitHub/audycje.com.pl/con...   
4   C:/Users/G/Documents/GitHub/audycje.com.pl/con...   
..                                                ...   
85  C:/Users/G/Documents/GitHub/audycje.com.pl/con...   
86  C:/Users/G/Documents/GitHub/audycje.com.pl/con...   
87  C:/Users/G/Documents/GitHub/audycje.com.pl/con...   
88  C:/Users/G/Documents/GitHub/audycje.com.pl/con...   
89  C:/Users/G/Documents/GitHub/audycje.com.pl/con...   

                                      summary_content  \
0   ---\n\n**Key Themes:**\nThe text revolves arou...   
1   **1. Key Themes**\n   - The text discusses the...   
2   ### Key Themes\n\nThe text discusses several i...   
3   ### Key Themes\nThe text discusses several key...   
4   ---\n\n### Key Themes:\n1.